In [42]:
import math
import treePlotter as tpl
import numpy
from sklearn import datasets
from sklearn.model_selection import train_test_split
import operator
import csv
from sklearn import datasets

def createDataSet():
    with open('PlayTennis.csv', 'r') as f:
        reader = csv.reader(f,delimiter=',')
        dataSet = list(reader)
    return dataSet[1:], dataSet[0]

In [43]:
classes = [0, 1, 2]
bc = datasets.load_iris()
a, b = bc.data, bc.target

c = numpy.column_stack([a, b])
print(a)
print(b)
print(c)


[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.

In [ ]:
def log(x: float)->float:
    if x == 0:
        return 0
    else:
        return math.log(x, 2)

def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet: # Le nombre d'occurrences pour chaque attribut
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob * log(prob) # log base 2
    return shannonEnt

In [ ]:
calcShannonEnt(a)

In [ ]:

def gain(unionSet, subsets):
    # input : data and disjoint subsets of it
    # output : information gain
    S = len(unionSet)
    # calculate impurity before split
    impurityBeforeSplit = entropy(unionSet)
    # calculate impurity after split
    weights = [len(subset) / S for subset in subsets]
    impurityAfterSplit = 0
    for i in range(len(subsets)):
        impurityAfterSplit += weights[i] * entropy(subsets[i])
    # calculate total gain
    totalGain = impurityBeforeSplit - impurityAfterSplit
    return totalGain

def entropy(dataSet):
    S = len(dataSet)
    if S == 0:
        return 0
    num_classes = [0 for i in classes]
    for row in dataSet:
        classIndex = list(classes).index(row[-1])
        num_classes[classIndex] += 1
    num_classes = [x / S for x in num_classes]
    ent = 0
    for num in num_classes:
        ent += num * log(num)
    return ent * -1



In [ ]:
entropy(c)

In [44]:
def splitDataSet(dataSet, axis, value):
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis] # Mettre dehors l'axe utilisé pour la subdivision
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [45]:
c = splitDataSet(a,0,'Overcast')
print(len(c),c)

0 []


In [46]:
splitDataSet(c,2,'weak')

[]

In [47]:
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1 # Dernière colonne contient la classe d'appartenance
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0; bestFeature = -1
    for i in range(numFeatures): # Pour chaque attribut
        featList = [example[i] for example in dataSet] # Liste des valeurs
        uniqueVals = set(featList) # Liste des valeurs sans doublons
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
        infoGain = baseEntropy - newEntropy # Calculer le gain
        if (infoGain > bestInfoGain): # Garder le meilleur gain
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature # rang de l'attribut: entier

In [48]:
ra = chooseBestFeatureToSplit(a)
print(ra)

AttributeError: 'numpy.ndarray' object has no attribute 'extend'

In [57]:
def splitDataSetNum(dataSet, axis, value, dir):
    # dir==0 for <=, 1 for >
    retDataSet = []
    for featVec in dataSet:
        if dir == 0:
            if featVec[axis] <= value:
                reducedFeatVec = featVec[:axis]
                reducedFeatVec.extend(featVec[axis+1:])
                retDataSet.append(reducedFeatVec)
        else:
            if featVec[axis] > value:
                reducedFeatVec = featVec[:axis]
                reducedFeatVec.extend(featVec[axis+1:])
                retDataSet.append(reducedFeatVec)
    return retDataSet

def chooseBestSplitNum(dataSet, labels):
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0; bestFeature = -1; bestSplitDict = {}
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        newEntropy = 0.0
        if isinstance(featList[0], int) or isinstance(featList[0], float):
            # Numerical feature
            mean = numpy.mean(featList)
            if mean == 0:
                continue
            gain0 = baseEntropy - calcShannonEnt(splitDataSetNum(dataSet, i, mean, 0))
            gain1 = baseEntropy - calcShannonEnt(splitDataSetNum(dataSet, i, mean, 1))
            if gain0>gain1 and gain0>bestInfoGain:
                bestInfoGain = gain0
                bestFeature = i
                bestSplitDict['index'] = i
                bestSplitDict['value'] = mean
                bestSplitDict['dir'] = 0 # <=
            elif gain1>gain0 and gain1>bestInfoGain:
                bestInfoGain = gain1
                bestFeature = i
                bestSplitDict['index'] = i
                bestSplitDict['value'] = mean
                bestSplitDict['dir'] = 1 # >
        else:
            for value in uniqueVals:
                subDataSet = splitDataSet(dataSet, i, value)
                prob = len(subDataSet)/float(len(dataSet))
                newEntropy += prob * calcShannonEnt(subDataSet)
            infoGain = baseEntropy - newEntropy # Calculer le gain
        if (infoGain > bestInfoGain): # Garder le meilleur gain
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature, bestSplitDict

# Rest same as ID3 algorithm but use splitDataSetNum and chooseBestSplitNum

In [58]:
rc=chooseBestFeatureToSplit(c)
print(rc)

IndexError: list index out of range

In [59]:
def majorityCnt(classList): # Retourner la classe avec la majorité de vote
    classCount={}
    for vote in classList:
        if vote not in classCount.keys(): classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(), key=lambda item: item[1], reverse=True)
    #print(sortedClassCount)
    return sortedClassCount[0][0]
majorityCnt(['no','no','no','yes'])

'no'

In [60]:
def createTree(dataSet,labels):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0] # Arrêter la decomposition (Exemples de la même classe)
    if len(dataSet[0]) == 1:
        return majorityCnt(classList) # Arrêter s'il ne reste qu'un seul attribut
    bestFeat = chooseBestSplitNum(dataSet, labels)
    bestFeatLabel = labels[bestFeat]
    print(bestFeatLabel)
    myTree = {bestFeatLabel:{}} # Initialiser le dictionnaire
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels.copy() # Copier chaque fois les noms des attributs
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value),subLabels)
    return myTree

In [61]:
dataSet, labels=createDataSet()
tr=createTree(dataSet, labels)
print(tr)

TypeError: list indices must be integers or slices, not tuple

In [54]:
tpl.createPlot(tr)

NameError: name 'tr' is not defined